# Tabular models

In [1]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [3]:
dep_var = 'salary' # 예측 대상 column
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'] # categorical value로 볼 columns
cont_names = ['age', 'fnlwgt', 'education-num'] # continuous value로 볼 columns
procs = [FillMissing, Categorify, Normalize] # processor. Fillmissing은 Null value를 무언가로 채워주고, Categorify는 Pandas에서 사용하는 categorical value로 바꿔주고, normalize는 각 feature들을 표준화를 시켜준다. 평균을 빼고, 표준편차로 나누어서 모든 feature value를 [0, 1]사이의 값으로 만듬. 

In [4]:
# 800:1000 라인은 validation으로 사용함.
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [5]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [6]:
data.show_batch(rows=100) # 데이터를 보니, 우리가 원하는 target이 50k 이상인지, 이하인지 찾는 모델을 만들어야한다.

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Local-gov,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.3362,-0.6152,-0.0312,>=50k
Private,Some-college,Divorced,Other-service,Not-in-family,White,False,-0.4095,-0.4597,-0.0312,<50k
Self-emp-not-inc,HS-grad,Never-married,Transport-moving,Unmarried,Black,False,-0.7027,0.3804,-0.4224,<50k
Local-gov,Some-college,Never-married,Farming-fishing,Own-child,White,False,-1.3624,0.4378,-0.0312,<50k
Private,Bachelors,Married-civ-spouse,Protective-serv,Husband,White,False,1.3496,-0.1831,1.1422,>=50k
Federal-gov,Bachelors,Divorced,Prof-specialty,Unmarried,White,False,0.6166,1.3224,1.1422,<50k
?,Some-college,Never-married,?,Own-child,White,False,-1.2891,-0.2447,-0.0312,<50k
Local-gov,Assoc-acdm,Married-civ-spouse,Farming-fishing,Husband,White,False,0.3968,0.1783,0.7511,<50k
Private,Some-college,Divorced,Sales,Own-child,White,False,0.4701,0.3185,-0.0312,<50k
State-gov,Some-college,Married-civ-spouse,Adm-clerical,Husband,Black,False,-0.9959,0.6385,-0.0312,<50k


### About Tabular Model 

![tabular](images/figs/tabularModel.png)

-> fastai에서 만든 tabular model은 위의 그림과 같다.
> column의 길이를 가지는 NN layer를 만들고.

> Dropout 0.0를 적용하며(default) (안쓴다는 의미)

> BatchNorm1D(tabular data라 data는 무조건 1D)를 적용하고

> ReLU를 activation function으로 사용하고

> hidden layer의 갯수는 layers List의 길이이다. (여기서는 [200, 100]이 들어오기 때문에, 2)

### Question.
- 파라미터로 들어가는 200, 100이 무슨 의미인지 정확하게 모르겠습니다.
- 추정하기로는 hidden layer의 node 갯수인것 같습니다. (nn.Sequential의 input이라서)
- 그러면 input layer와 output layer의 node수는 자동으로 원하는 task에 따라 맞춰지는 것인가요?
- Input layer는 data column수이고, output layer는 target에 따라 자동으로 결정되는 건지..? 궁금합니다.

*layers=[200, 100] 부분은 layers 중에 100번 부터 200까지 뽑는 것임*

*documentation 보니 : The sizes of the blocks are given in layers 라고 함. Hidden layer의 노드로 맞음* 

*네 tabular model은 자동으로 model을 해 주는 것임*

In [7]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy) # accuracy를 통해 target을 잘 맞췄는지 판단. 지금은 binary classification 문제를 푸는것이 됨.

#### 학습을 더 시키니, Accuracy가 84%까지 올라가는 것을 볼 수 있다. 근데 10 epoch까지 돌리니 소폭 감소하는 것으로 보아 overfitting이 일어나는 것 같다. (validation loss가 안떨어지고 조금씩 올라오는 것도 확인됨.)

In [11]:
#learn.fit(1, 1e-2)
learn.fit(10, 1e-3) # 학습을 좀더 해보았다.

epoch,train_loss,valid_loss,accuracy,time
0,0.353470,0.375416,0.825000,00:03
1,0.361984,0.374294,0.825000,00:03
2,0.355286,0.375979,0.835000,00:03
3,0.357826,0.379274,0.830000,00:03
4,0.350756,0.369321,0.840000,00:02
5,0.355748,0.369593,0.830000,00:02
6,0.348889,0.364290,0.845000,00:02
7,0.345793,0.361450,0.845000,00:02
8,0.347361,0.362146,0.840000,00:02
9,0.347063,0.359788,0.840000,00:02


## Inference

In [9]:
row = df.iloc[0] # 첫번째 데이터를 test

In [10]:
learn.predict(row) # category를 잘 판단하는 것을 알 수 있다. 결과로 0.4039, 0.5961이 나오기 때문에, class 1로 판단하고, 그것은 >=50k이다.

(Category >=50k, tensor(1), tensor([0.4039, 0.5961]))

## Comment:

- 보통 tabular data는 xgboost를 많이 사용하는데, (in-class question으로도 나옴.) NN을 통해서 간단하게 prediction 되는것이 신기했다.]
- 물론 design에 따라 당연히 바뀔것같은데, fastai에서 default로 널리 쓰이는 ML 방법대신, 간단한 DL 모델을 제공하는게 사실 신기했다.
- adult dataset은 너무 흔한 옛날 데이터인데 77.5%밖에 안나오는걸 보면 xgboost가 더 낫지 않을까 싶기도 한데, parameter tuning을 해보면 달라질 것 같다.
- 고쳐서 돌려보니 84%까지 올라갔다.